In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
pd.set_option("display.max_rows", 101)
pd.set_option("display.max_columns", 101)
%matplotlib inline

In [2]:
train=pd.read_csv("train/train_data.csv")
label=pd.read_csv("train/train_label.csv")
test=pd.read_csv("test/test_data.csv")

In [3]:
#入力がずれているので調整
train=train.iloc[1:,:].reset_index(drop=True)

In [4]:
def word_feature(sentence, tokenizer):
    tokens = tokenizer.tokenize(sentence)
    bases = [str(t.base_form) for t in tokens]
    return bases

def chara_feature(sentence, tokenizer):
    bases = [i for i in sentence]
    return bases

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from janome.tokenizer import Tokenizer
tokenizer = Tokenizer()
feature_func = lambda x: word_feature(x, tokenizer)
texts = pd.concat([train['text'],test["text"]],axis=0)
#word_counter = CountVectorizer(min_df=3, tokenizer=feature_func,max_features=100)
#word_counter =TfidfVectorizer(max_features=25000, ngram_range=1, tokenizer=feature_func)
tfidf = TfidfVectorizer(tokenizer=feature_func,max_features=500)
tmp = tfidf.fit_transform(texts)

feature_func = lambda x: chara_feature(x, tokenizer)
tfidf = TfidfVectorizer(tokenizer=feature_func,max_features=500)
tmp2 = tfidf.fit_transform(texts)

In [12]:
train_co=tmp[:train.shape[0]].todense()
test_co=tmp[train.shape[0]:].todense()
train_co2=tmp2[:train.shape[0]].todense()
test_co2=tmp2[train.shape[0]:].todense()

In [13]:
from sklearn import preprocessing
tmp=pd.concat([train,test],axis=0)
le = preprocessing.LabelEncoder()

le.fit(tmp["place"].unique())
place2=le.transform(tmp["place"]) 

le.fit(tmp["open_type"].fillna("nan").unique())
open_type2=le.transform(tmp["open_type"].fillna("nan")) 

le.fit(tmp["open_hours_st"].fillna("nan").unique())
open_hours_st2=le.transform(tmp["open_hours_st"].fillna("nan"))

le.fit(tmp["open_days_ed"].fillna("nan").unique())
open_days_ed2=le.transform(tmp["open_days_ed"].fillna("nan")) 

In [14]:
train["placenum"]=place2[:train.shape[0]]
test["placenum"]=place2[train.shape[0]:]
train["open_type2"]=open_type2[:train.shape[0]]
test["open_type2"]=open_type2[train.shape[0]:]
train["open_hours_st2"]=open_hours_st2[:train.shape[0]]
test["open_hours_st2"]=open_hours_st2[train.shape[0]:]
train["open_days_ed2"]=open_days_ed2[:train.shape[0]]
test["open_days_ed2"]=open_days_ed2[train.shape[0]:]

In [20]:
#mlp_train=pd.concat([train[["placenum","open_allday",'open_type2',
#       'open_hours_st2', 'open_days_ed2']],pd.DataFrame(train_co),pd.DataFrame(train_co2)],axis=1)
#mlp_test=pd.concat([test[["placenum","open_allday",'open_type2',
#       'open_hours_st2', 'open_days_ed2']],pd.DataFrame(test_co),pd.DataFrame(test_co2)],axis=1)
mlp_train=pd.concat([pd.DataFrame(train_co),pd.DataFrame(train_co2)],axis=1)
mlp_test=pd.concat([pd.DataFrame(test_co),pd.DataFrame(test_co2)],axis=1)

In [17]:
#lavel変換
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(label["Sports_Resort"].unique())
label2=le.transform(label["Sports_Resort"]) 

In [18]:
import keras
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, GlobalAveragePooling2D,Activation
from keras.optimizers import SGD, Adam
#use mlp for staking
def build_mlp_model(input_dim):
    model_input = Input(shape=(input_dim,))
    #model_input = Input(shape=(input_dim,), sparse=True)
    x = Dense(192, activation='relu')(model_input)
    x = Dense(64, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(8)(x)
    output=Activation('softmax')(x)
    model = keras.Model(model_input, output)
    #model.compile(
    #    loss='mean_squared_error',
    #    optimizer=keras.optimizers.Adam(lr=3e-3)
    #)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

/Users/rmizuta/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [21]:
model=build_mlp_model(mlp_train.shape[1])

In [22]:
from keras.utils import np_utils
Y = np_utils.to_categorical(label2)

In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1005)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 192)               193152    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                12352     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 520       
_________________________________________________________________
activation_1 (Activation)    (None, 8)                 0         
Total params: 210,184
Trainable params: 210,184
Non-trainable params: 0
_________________________________________________________________


In [77]:
from keras import backend as K
def reset_weights(model):
        session = K.get_session()
        for layer in model.layers: 
            if hasattr(layer, 'kernel_initializer'):
                layer.kernel.initializer.run(session=session)
                
#for stacking
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(mlp_train, Y, stratify=Y, test_size=0.2, random_state=942)
train_mlpfeature=np.empty([0,8])
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,random_state=None, shuffle=False)
for train_index, test_index in kf.split(mlp_train):
    
    X_train, y_train = mlp_train.iloc[train_index,:], Y[train_index]
    X_test, y_test = mlp_train.iloc[test_index,:], Y[test_index]
    
    reset_weights(model)
            
    model.fit(X_train, y_train,
              batch_size=32,
              epochs=10,
              validation_data=(X_test, y_test),
              shuffle=True)
    
    train_mlpfeature=np.append(train_mlpfeature,model.predict(X_test),axis=0)
    

Train on 3573 samples, validate on 894 samples
Epoch 1/10
3573/3573 [==============================] - 1s 182us/step - loss: 1.5669 - acc: 0.4344 - val_loss: 1.2016 - val_acc: 0.5347
Epoch 2/10
3573/3573 [==============================] - 1s 191us/step - loss: 0.8594 - acc: 0.6991 - val_loss: 0.6821 - val_acc: 0.7919
Epoch 3/10
3573/3573 [==============================] - 1s 211us/step - loss: 0.5782 - acc: 0.7996 - val_loss: 0.5565 - val_acc: 0.8076
Epoch 4/10
3573/3573 [==============================] - 0s 133us/step - loss: 0.4541 - acc: 0.8463 - val_loss: 0.5069 - val_acc: 0.8322
Epoch 5/10
3573/3573 [==============================] - 0s 131us/step - loss: 0.3976 - acc: 0.8631 - val_loss: 0.4917 - val_acc: 0.8333
Epoch 6/10
3573/3573 [==============================] - 0s 134us/step - loss: 0.3240 - acc: 0.8892 - val_loss: 0.4974 - val_acc: 0.8412
Epoch 7/10
3573/3573 [==============================] - 1s 141us/step - loss: 0.2799 - acc: 0.9004 - val_loss: 0.5306 - val_acc: 0.8311
E

In [78]:
#train_mlpfeature=model.predict(mlp_train)
test_mlpfeature=model.predict(mlp_test)

In [79]:
train2=pd.concat([train[["placenum","open_allday",'open_type2',
       'open_hours_st2', 'open_days_ed2']],pd.DataFrame(train_co),pd.DataFrame(train_co2),pd.DataFrame(train_mlpfeature)],axis=1)
test2=pd.concat([test[["placenum","open_allday",'open_type2',
       'open_hours_st2', 'open_days_ed2']],pd.DataFrame(test_co),pd.DataFrame(test_co2),pd.DataFrame(test_mlpfeature)],axis=1)

In [80]:
train_mlpfeature.shape

(4467, 8)

In [81]:
len(label2)

4467

In [82]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedShuffleSplit
import copy

params={}

params={
    'objective': 'multiclass',
    'num_class': 8,
    'random_state' : 1,
    #"objective": "rmse",
    #"eval_metric": "rmse",
    #"objective": "binary",
    "metric": "multi_error",
    #'num_leaves': 50,
    }

df_train=train2
df_label=label2

p_test=0
p_eval=0
kfold = 5
score=0
sss = StratifiedShuffleSplit(n_splits=kfold, test_size=0.2, random_state=94)
for i, (train_index, test_index) in enumerate(sss.split(df_train, df_label)):
    print('[Fold %d/%d]' % (i + 1, kfold))
    
    X_train, X_valid = df_train.iloc[train_index,:], df_train.iloc[test_index,:]
    y_train, y_valid = df_label[train_index], df_label[test_index]
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_valid, label=y_valid)
    
    #watchlist = [(dtrain, 'train'),(dvalid, 'eval')]
    #watchlist = [(dvalid, 'eval')]
    #bst = lgb.train(params, dtrain, num_boost_round=1000,valid_sets=[dtrain, dvalid],early_stopping_rounds=50,verbose_eval=10)
    bst = lgb.train(params, dtrain, num_boost_round=1000,valid_sets=[dvalid],early_stopping_rounds=50,verbose_eval=10)

    #d_test = pred_df
    #p_test+=bst.predict(d_test)
    p_test+=bst.predict(test2)
    score+=bst.best_score["valid_0"]['multi_error']
    
#p_test=p_test/5
#p_test = np.argmax(p_test, axis=1)

[Fold 1/5]
Training until validation scores don't improve for 50 rounds.
[10]	valid_0's multi_error: 0.193512
[20]	valid_0's multi_error: 0.189038
[30]	valid_0's multi_error: 0.192394
[40]	valid_0's multi_error: 0.182327
[50]	valid_0's multi_error: 0.180089
[60]	valid_0's multi_error: 0.177852
[70]	valid_0's multi_error: 0.178971
[80]	valid_0's multi_error: 0.180089
[90]	valid_0's multi_error: 0.180089
[100]	valid_0's multi_error: 0.182327
Early stopping, best iteration is:
[52]	valid_0's multi_error: 0.177852
[Fold 2/5]
Training until validation scores don't improve for 50 rounds.
[10]	valid_0's multi_error: 0.177852
[20]	valid_0's multi_error: 0.175615
[30]	valid_0's multi_error: 0.176734
[40]	valid_0's multi_error: 0.174497
[50]	valid_0's multi_error: 0.170022
[60]	valid_0's multi_error: 0.168904
[70]	valid_0's multi_error: 0.177852
[80]	valid_0's multi_error: 0.17226
[90]	valid_0's multi_error: 0.176734
Early stopping, best iteration is:
[46]	valid_0's multi_error: 0.168904
[Fold 3

In [83]:
#MLPあり、なしより下がってる？？？
score/5

0.16711409395973154

In [35]:
#MLPなし
score/5

0.16308724832214766

In [19]:
#charのtfidfなし
score/5

0.2073825503355705

In [308]:
#preds=bst.predict(pd.DataFrame(test_co))
#preds=bst.predict(test2)
p_test=p_test/5
preds2=np.argmax(p_test,axis=1)
result=le.inverse_transform(preds2)
submit=pd.DataFrame()
submit["id"]=test["id"]
submit["Park"]=result

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [309]:
submit.to_csv("park9.csv",index=False)